In [183]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.metrics import mean_squared_error
import scipy.optimize as opt
from scipy.optimize import differential_evolution
import concurrent.futures
from tradingview_screener import Query
import rookiepy


with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")

pd.set_option('display.max_rows', None)


In [184]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [185]:
    
#TODO FIX SCREENER 
def screen_stocks():
    # Get cookies for TradingView session
    cookies = rookiepy.to_cookiejar(rookiepy.chrome(['.tradingview.com']))
    
    # Fetch stock data
    _, df = Query().select('close','change', 'EMA30').where('close' < 75 ).limit(1000).get_scanner_data(cookies=cookies)
    
    # Apply filters
    filtered_df = df[(df['change'] < -2.5) & (df['EMA30'] > 2) & df['close'] < 55]

    return filtered_df

def filter_stocks(rolling_change_period): 
    filtered_stocks = set()

    for index, stock in DOWJ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'], days=rolling_change_period)
        if today_change is not None and rolling_avg is not None:
            if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
                filtered_stocks.add(stock['Symbol'])

    for index, stock in NASDAQ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'], days=rolling_change_period)
        if today_change is not None and rolling_avg is not None:
            if (today_change <= -2.00) and (rolling_avg > 0.00) and (get_current_stock_price(stock['Symbol']) < 150): 
                filtered_stocks.add(stock['Symbol'])

    return filtered_stocks

def get_price_change_and_rolling_avg(ticker: str, days: int):
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days+10)
        
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            return None, None
        
        data = data.sort_index()
        current_price = get_current_stock_price(ticker)

        data['Price_Change'] = ((current_price - data['Close'].shift(1)) / data['Close'].shift(1)) * 100
        today_price_change = data['Price_Change'].iloc[-1]

        rolling_avg = data['Price_Change'].rolling(window=min(days, len(data))).mean().iloc[-1]

        return today_price_change, rolling_avg
    
    except Exception as e:
        print(f"Error occurred for ticker {ticker}: {e}")
        return None, None

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def get_option_chain(api_key: str, secret_key: str, ticker: str, expiration_date: datetime):
    expiration_str = expiration_date.strftime("%Y-%m-%d")  # Convert datetime to string
    
    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{ticker}?feed=indicative&limit=100&expiration_date={expiration_str}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        option_chain = data.get("snapshots", {})

        if not option_chain:
            return None

        parsed_data = []
        for symbol, details in option_chain.items():
            expiration_start = len(symbol) - 15
            option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
            strike_price = int(symbol[expiration_start+7:]) / 1000  

            greeks = details.get("greeks", {}) or {}
            latest_quote = details.get("latestQuote", {})

            parsed_data.append({
                "symbol": ticker,
                "expiration_date": expiration_str,  # Use the formatted string
                "option_type": option_type,
                "strike_price": strike_price,
                "delta": greeks.get("delta"),
                "gamma": greeks.get("gamma"),
                "rho": greeks.get("rho"),
                "theta": greeks.get("theta"),
                "vega": greeks.get("vega"),
                "implied_volatility": details.get("impliedVolatility"),
                "ask_price": latest_quote.get("ap"),
                "ask_size": latest_quote.get("as"),
                "bid_price": latest_quote.get("bp"),
                "bid_size": latest_quote.get("bs"),
            })

        return pd.DataFrame(parsed_data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None
    
    

def gbm(s0, mu, sigma, deltaT, dt):
    """
    Models a stock price S(t) using the Wiener process W(t) as
    `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
    Arguments:
        s0: Initial stock price, default 100
        mu: 'Drift' of the stock (upwards or downwards), default 0.2
        sigma: 'Volatility' of the stock, default 0.68
        deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
        dt: The granularity of the time-period, default 0.1
    
    Returns:
        time_vector: array of time steps
        s: array with the simulated stock prices over the time-period deltaT
    """
    n_step = int(deltaT / dt)  # Number of time steps
    time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
    # Wiener process: cumulative sum of random normal increments
    random_increments = np.random.normal(0, np.sqrt(dt), size=n_step)
    weiner_process = np.cumsum(random_increments)
    
    # Stock price simulation
    stock_var = (mu - (sigma**2 / 2)) * time_vector
    s = s0 * np.exp(stock_var + sigma * weiner_process)
    
    return s



def archive_objective(params, real_prices, s0):
    """Objective function for optimization."""
    mu, sigma = params  # Unpack parameters
    gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
    return mean_squared_error(real_prices, gbm_prices)

def archive_optimize_gbm(symbol: str, training_period: str, bin_length: int):
    """
    Optimize μ and σ over multiple time bins, weighting recent periods more.
    """
    # Fetch real stock data (past 5 years)
    stock_data = yf.download(symbol, period=training_period, interval="1d")
    real_prices = stock_data["Close"].dropna().values

    num_bins = len(real_prices) // bin_length
    weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    mu_values, sigma_values, mses = [], [], []

    for i in range(num_bins):
        bin_prices = real_prices[i * bin_length : (i + 1) * bin_length]
        s0 = bin_prices[0]

        bounds = [(-0.3, 0.3), (0.001, 0.35)]

        result = differential_evolution(archive_objective, bounds, args=(bin_prices, s0))
        best_mu, best_sigma = result.x
        best_mse = result.fun

        mu_values.append(best_mu)
        sigma_values.append(best_sigma)
        mses.append(best_mse)

    weight_sum = np.sum(weights)
    avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    return avg_mu, avg_sigma


def compute_score(contract):
    """Compute the weighted score for a contract."""
    return (0.25 * contract['profitability_percent'] +
            0.25 * contract['percent_return'] +
            0.50 * contract['sharpe_ratio'])

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score."""
    contracts['score'] = contracts.apply(compute_score, axis=1)
    
    contracts = contracts.sort_values(by='score', ascending=False)
    # optimal_contract = contracts.loc[contracts['score'].idxmax()]
    
    return contracts


In [186]:
simulation_attempts = 200
optimizer_training_period = "2y"
bin_length = 20
rolling_change_period = 15
expiration_date = datetime(year=2025, month=2, day=7) 
all_options = pd.DataFrame(columns=['symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 'ask_size', 'bid_price', 'bid_size'])
candidates = ["AAPL", "AMD"]
# filter_stocks(rolling_change_period=rolling_change_period)

# t-bill 3-month rate: 4.19%, inflation rate: 2.9% -> scaled to weekly
risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100

print(candidates)

for symbol in candidates:
    option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)
    put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)

    if option_chain is None or option_chain.empty:
        continue 

    price = get_current_stock_price(symbol)
    optimized_mu, optimized_sigma = archive_optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length)

    profitability_chances = []
    percent_returns = []

    for index, contract in put_chain.iterrows():
        count = 0
        strike_price = contract['strike_price']

        for i in range(simulation_attempts):
            prices = gbm(s0=price, mu=optimized_mu, sigma=optimized_sigma, 
                deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), dt=1)
        
            if prices[-1] > strike_price:
                count += 1

        profitability_chance = (count / simulation_attempts) * 100
        profit = (contract['bid_price']*contract['bid_size'] + contract['ask_price']*contract['ask_size']) / (contract['ask_size'] + contract['bid_size'])
        percent_return = (profit / (strike_price)) * 100

        profitability_chances.append(profitability_chance)
        percent_returns.append(percent_return)

    put_chain['profitability_percent'] = profitability_chances
    put_chain['percent_return'] = percent_returns
    put_chain['expected_value'] = put_chain['profitability_percent'] * put_chain['percent_return']
    put_chain['current_price'] = price

    if put_chain['percent_return'].std() != 0:
        put_chain['sharpe_ratio'] = (put_chain['percent_return'] - risk_free_rate) / put_chain['percent_return'].std()
    else:
        put_chain['sharpe_ratio'] = 0  # Avoid division by zero

    all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)



['AAPL', 'AMD']


,symbol,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size
0,AAPL,2025-02-07,Call,270.0,NaN,NaN,NaN,NaN,NaN,NaN,0.03,1292,0.00,0
1,AAPL,2025-02-07,Put,230.0,-0.3822,0.0589,-0.0074,-0.4008,0.0801,0.3080,1.77,47,1.74,365
2,AAPL,2025-02-07,Call,230.0,0.6136,0.0568,0.0114,-0.4464,0.0804,0.3209,3.77,713,3.63,907
3,AAPL,2025-02-07,Call,240.0,0.1130,0.0293,0.0021,-0.2122,0.0403,0.3115,0.36,625,0.35,30
4,AAPL,2025-02-07,Put,150.0,NaN,NaN,NaN,NaN,NaN,NaN,0.03,631,0.00,0
5,AAPL,2025-02-07,Call,192.5,NaN,NaN,NaN,NaN,NaN,NaN,39.48,94,39.30,6
6,AAPL,2025-02-07,Put,207.5,-0.0071,0.0019,-0.0001,-0.0347,0.0042,0.5038,0.03,335,0.02,372
7,AAPL,2025-02-07,Put,140.0,NaN,NaN,NaN,NaN,NaN,NaN,0.01,630,0.00,0
8,AAPL,2025-02-07,Call,235.0,0.3158,0.0559,0.0059,-0.3854,0.0747,0.3026,1.29,205,1.27,270
9,AAPL,2025-02-07,Call,202.5,0.9274,0.0062,0.0152,-0.5344,0.0290,1.0597,30.33,92,29.83,14


[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [ ]:
# TODO
"""
- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer
- Find better metric for optimizer
- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins
"""

contracts = select_optimal_contract(all_options).iloc[:5]
display(contracts)


,symbol,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size,profitability_percent,percent_return,expected_value,current_price,sharpe_ratio,score
27,AMD250207P00108000,2025-02-07,Put,108.0,-0.1989,0.0216,-0.0020,-0.5978,0.0298,1.2101,1.52,250,1.50,163,100.0,1.400099,140.009865,117.76,0.548951,25.624500
26,AMD250207P00107000,2025-02-07,Put,107.0,-0.1741,0.0201,-0.0018,-0.5452,0.0274,1.1985,1.29,425,1.23,9,100.0,1.204445,120.444464,117.76,0.470904,25.536563
28,AMD250207P00109000,2025-02-07,Put,109.0,-0.2235,0.0231,-0.0023,-0.6409,0.0319,1.2120,1.77,52,1.75,141,99.0,1.610448,159.434378,117.76,0.632861,25.469043
29,AMD250207P00110000,2025-02-07,Put,110.0,-0.2508,0.0244,-0.0026,-0.6891,0.0340,1.2234,2.08,51,2.06,50,98.5,1.881908,185.367957,117.76,0.741149,25.466051
24,AMD250207P00105000,2025-02-07,Put,105.0,-0.1319,0.0168,-0.0013,-0.4512,0.0228,1.1920,0.91,50,0.86,523,100.0,0.823203,82.320286,117.76,0.318823,25.365212


In [ ]:
# Archive

# def gbm_vs_real_graph(symbol, mu, sigma, period):
#     stock_data = yf.download(symbol, period=period, interval="1d")
#     real_prices = stock_data["Close"].dropna().values
#     time_steps = np.arange(len(real_prices))


#     gbm_path = gbm(s0 = real_prices[0], mu=mu, sigma=sigma, deltaT=len(real_prices), dt=1)
#     plt.figure(figsize=(10, 5))
#     plt.plot(time_steps, real_prices, label="Real Prices", color="blue")
#     plt.plot(time_steps, gbm_path, label="GBM Simulated", linestyle="dashed", color="red")
    
#     plt.xlabel("Time (Days)")
#     plt.ylabel("Price")
#     plt.title(f"GBM vs Real Prices for {symbol}")
#     plt.legend()
#     plt.grid()
#     plt.show()

# def multithread_optimize_bin(bin_prices, bin_size, weights, i):
#     s0 = bin_prices[0]

#     # Define the bounds for optimization
#     bounds = [(-0.3, 0.3), (0.001, 0.30)]

#     # Run the optimizer for the bin
#     result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#     best_mu, best_sigma = result.x
#     best_mse = result.fun

#     print(f"Bin {i+1}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")
#     return best_mu, best_sigma, best_mse

# def multithread_optimize_gbm(symbol): 
    # """
    # Optimize μ and σ over multiple time bins, weighting recent periods more.
    # """
    # # Fetch real stock data (past 2 years)
    # stock_data = yf.download(symbol, period="2y", interval="1d")
    # real_prices = stock_data["Close"].dropna().values

    # # Split into bins of 20 trading days
    # bin_size = 20
    # num_bins = len(real_prices) // bin_size
    # weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    # # Initialize containers for results
    # mu_values, sigma_values, mses = [], [], []

    # # Use concurrent.futures for parallel processing of bins
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #     futures = []
    #     for i in range(num_bins):
    #         bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
    #         futures.append(executor.submit(optimize_bin, bin_prices, bin_size, weights, i))
        
    #     for future in concurrent.futures.as_completed(futures):
    #         best_mu, best_sigma, best_mse = future.result()
    #         mu_values.append(best_mu)
    #         sigma_values.append(best_sigma)
    #         mses.append(best_mse)

    # # Compute weighted averages
    # weight_sum = np.sum(weights)
    # avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    # avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    # print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    # return avg_mu, avg_sigma